# 1  定义工具

In [21]:
import os

from langchain_community.tools.tavily_search import TavilySearchResults
import dotenv
from pywin.framework.toolmenu import tools

dotenv.load_dotenv()
# 定义 AVILY_KEY 密钥
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
# 查询 Tavily 搜索 API
search = TavilySearchResults(max_results=1,description="用于搜索实时网络信息，特别适合查询天气、新闻等实时数据")
# 执行查询
res = search.invoke("今天珠海天气怎么样")
print(res)

[{'title': '珠海 - 中国气象局-天气预报-城市预报', 'url': 'https://weather.cma.cn/web/weather/59488', 'content': '|  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 25.5℃ | 25.8℃ | 27.6℃ | 27℃ | 25.1℃ | 24.5℃ | 24.4℃ | 24.6℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 3.2m/s | 3.2m/s | 3.3m/s | 2.6m/s | 2.1m/s | 3m/s | 2.4m/s | 1.8m/s |\n| 风向 | 东北风 | 东北风 | 东南风 | 东南风 | 东南风 | 东南风 | 东北风 | 东北风 |\n| 气压 | 1014.2hPa | 1015hPa | 1012.7hPa | 1011.8hPa | 1012.6hPa | 1013.1hPa | 1012.6hPa | 1011.7hPa | [...] |  |  |  |  |  |  |  |  |  |\n ---  ---  ---  --- \n| 时间 | 08:00 | 11:00 | 14:00 | 17:00 | 20:00 | 23:00 | 02:00 | 05:00 |\n| 天气 |\n| 气温 | 25.6℃ | 26.8℃ | 27.6℃ | 25.5℃ | 25.4℃ | 25.4℃ | 24℃ | 24.4℃ |\n| 降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 | 无降水 |\n| 风速 | 7.5m/s | 3.3m/s | 2.9m/s | 2.2m/s | 2.6m/s | 3.3m/s | 3.1m/s | 2.3m/s |\n| 风向 | 东北风 | 东北风 | 东北风 | 东南风 | 东南风 | 东北风 | 东北风 | 东北风 |\n| 气压 | 10

# 2 定义Retriever

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import dotenv
dotenv.load_dotenv()

# 1. 提供一个大模型
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY1")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

embedding_model = OpenAIEmbeddings()

# 2.加载HTML内容为一个文档对象
loader = WebBaseLoader("https://zh.wikipedia.org/wiki/%E7%8C%AB")
docs = loader.load()
#print(docs)

# 3.分割文档
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

documents = splitter.split_documents(docs)

# 4.向量化 得到向量数据库对象
vector = FAISS.from_documents(documents, embedding_model)

# 5.创建检索器
retriever = vector.as_retriever()

# 测试检索结果
# print(retriever.invoke("猫的特征")[0])

USER_AGENT environment variable not set, consider setting it to identify your requests.


# 3 创建工具、工具集

In [22]:
from langchain.tools.retriever import create_retriever_tool

# 创建一个工具来检索文档
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="retriever_tool",
    description="猫的相关介绍,之和猫有关时候选",
)

# 构建工具集
tools = [search, retriever_tool]

# 4 语言模型调用工具

In [23]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 获取大模型
model = ChatOpenAI(model="gpt-4o-mini")

# 模型绑定工具
model_with_tools = model.bind_tools(tools)

# 根据输入自动调用工具
messages = [HumanMessage(content="今天珠海天气怎么样")]
response = model_with_tools.invoke(messages)
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '珠海天气'}, 'id': 'call_2bxwN3V7x6HnGl9jkSN9qBqe', 'type': 'tool_call'}]


# 5 创建Agent程序(使用通用方式)

In [18]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

print(prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), MessagesPlaceholder(variable_name='agent_scratchpad')]


In [24]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 创建Agent对象
agent = create_tool_calling_agent(model, tools, prompt)

# 创建AgentExecutor对象
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

# 6 运行Agent

In [8]:
print(agent_executor.invoke({"input": "猫的特征"}))



> Entering new AgentExecutor chain...

Invoking: `wiki_search` with `{'query': '猫的特征'}`


感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[51]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[52][53]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。

貓爪[编辑]
貓的爪子尖

In [26]:
print(agent_executor.invoke({"input": "今天lol比赛"}))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'LOL比赛 2023年10月28日 赛事安排'}`


[{'title': '英雄聯盟2023賽季世界大賽- 維基百科', 'url': 'https://zh.wikipedia.org/wiki/%E8%8B%B1%E9%9B%84%E8%81%94%E7%9B%9F2023%E8%B5%9B%E5%AD%A3%E5%85%A8%E7%90%83%E6%80%BB%E5%86%B3%E8%B5%9B', 'content': '比赛时间：2023年11月2日至11月19日（UTC+9）\n 比赛场馆： 釜山社稷室内体育馆（四分之一决赛及半决赛）、 首尔高尺天空巨蛋（总决赛）\n 淘汰赛共分为四分之一决赛，半决赛与总决赛三个阶段\n 8支队伍根据抽签产生四分之一决赛对手；来自瑞士轮阶段同一分组晋级的队伍将不会分在同一半区且不能成为四分之一决赛对手，即启用半区规避原则\n 每阶段比赛均为BO5比赛，胜者晋级下一阶段，直至總决赛\n\n|  |  |  |\n --- \n| 四分之一决赛 | 半决赛 | 总决赛 |\n| NRG Esports | 0 |\n| Weibo Gaming | 3 |\n| Weibo Gaming | 3 |\n| Bilibili Gaming | 2 |\n| Gen.G | 2 |\n| Bilibili Gaming | 3 |\n| Weibo Gaming | 0 |\n| T1 | 3 |\n| JD Gaming | 3 |\n| KT Rolster | 1 |\n| JD Gaming | 1 |\n| T1 | 3 |\n| LNG Esports | 0 |\n| T1 | 3 | [...] ## 瑞士轮\n\n[编辑]\n\n瑞士轮赛制\n\n 比赛时间：10月19日至10月29日（UTC+9）\n 比赛场馆： 首尔KBS竞技馆\n 由16支队伍进行五轮瑞士轮比赛\n  + 第一轮通过抽签决定对阵队伍以Bo1的形式比赛，此后每一轮相同战绩的队伍互相对决\n  + 取得3胜的队伍晋级，取得3败的队伍淘汰\n  + 

In [27]:
print(agent_executor.invoke({"input": "今天猫的特征"}))



> Entering new AgentExecutor chain...

Invoking: `retriever_tool` with `{'query': '猫的特征'}`


感官[编辑]
貓的感官適於狩獵。在哺乳類動物中，貓的聽覺、視覺、嗅覺、味覺、觸覺極敏銳。

視覺[编辑]
貓的瞳孔能縮得如線般狹小
貓的瞳孔變化
貓在晝間視覺縱不及人類，夜視能力與追蹤視覺上之活動物件卻相當出色，夜視能力是人類的六倍，雖然綜合色彩計算整體視覺系數則僅及人類的十分之一。貓的眼睛具有微光觀察能力，即使只有微弱月光都可分辨物件，有關光線入貓眼後可放大40至50倍處理，令貓具有夜間活動的能力，即使在黑暗的地下室貓咪依然能活動自如。強光下，貓會將瞳孔縮得如線般狹小，以減少對視網膜的傷害，但視野會因而縮窄。由於貓眼具備高幀與高分辨率視覺，故此電視機極微細之動靜對貓而言亦成逐格跳躍之畫面。[51]
另外，不只是光線會影響貓的瞳孔，感情也會。一般而言貓放鬆的時候瞳孔會縮小，而緊張時會放大。
貓的視網膜背面有一層藍綠色如熒光一般的薄膜（Tapetum Lucidum），可增加在暗處的視力。閃光中，貓眼能呈現各式各樣顔色。如同多數食肉動物，貓眼長在臉上朝正前方，賦予其遼闊的視野，單一貓眼視野為100度，雙眼視野為285度，加上頸旋靈活，總計視野比人類雙眼僅100度靈活得多。不過，貓僅能聚焦其前30厘米至3米之物件，相對人類而言屬大近視。
貓對三原色的辨識力很差，相對人類而言，貓有黃藍色盲且紅綠色辨成灰黃色，與狗相似。關於貓的夜視能力，生物學家發現牛磺酸對貓的視力起了很大作用，貓本身不能合成牛磺酸，必須由外攝取。缺乏牛磺酸，會使視力及夜視能力變差，影響夜間活動。據生物學家稱，貓經常捕鼠，是因老鼠體內含牛磺酸。[52][53]
紫外線可以通過貓的晶状体。
當四周光線微弱，貓會用感覺毛來改善行動力與感知能力。感覺毛主要分布於鼻子兩側、下巴、雙眼上方、兩頰也有數根。感覺毛可感受非常微弱的空氣波動，視野不清時也能協助辨識阻礙位置。鬍鬚尖端與雙耳連成一線，恰是身體能通過障礙的最小範圍，故貓可在黑夜中快速判斷地形能否通過。
貓有第三眼瞼，當貓眼睑張開時，眨眼時第三眼瞼會從旁稍微遮蓋眼睛。若貓生病，或是睡眠，笑著，此眼皮會縮回一部分。若貓長時間嶄露第三眼瞼，表示它的健康有問題。

貓爪[编辑]
貓的

# 7 添加记忆

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain_core.chat_history import BaseChatMessageHistory

from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# 调取指定session_id对应的memory
def get_session_history(session_id: str) -> BaseChatMessageHistory:

    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    runnable=agent_executor,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

response = agent_with_chat_history.invoke(
    {"input": "Hi，我的名字是Cyber"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
嗨，Cyber！很高兴见到你！有什么我可以帮助你的吗？

> Finished chain.
{'input': 'Hi，我的名字是Cyber', 'chat_history': [], 'output': '嗨，Cyber！很高兴见到你！有什么我可以帮助你的吗？'}


In [29]:
response = agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    config={"configurable": {"session_id": "123"}},
)

print(response)



> Entering new AgentExecutor chain...
你的名字是Cyber。需要我帮你做什么吗？

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [HumanMessage(content='Hi，我的名字是Cyber', additional_kwargs={}, response_metadata={}), AIMessage(content='嗨，Cyber！很高兴见到你！有什么我可以帮助你的吗？', additional_kwargs={}, response_metadata={})], 'output': '你的名字是Cyber。需要我帮你做什么吗？'}


In [30]:
response = agent_with_chat_history.invoke(
    {"input": "我叫什么名字?"},
    config={"configurable": {"session_id": "4566"}},
)
print(response)



> Entering new AgentExecutor chain...
抱歉，我无法知道你的名字。你可以告诉我你的名字，或者我可以帮助你找到其他想知道的信息。

> Finished chain.
{'input': '我叫什么名字?', 'chat_history': [], 'output': '抱歉，我无法知道你的名字。你可以告诉我你的名字，或者我可以帮助你找到其他想知道的信息。'}
